### Lab 1.3: Multi-Class Linear Classifier

In this lab you will explore multi-class classification and evaluate model generalization using a [dataset for heart disease prediction from the UCI ML repository](https://archive.ics.uci.edu/dataset/45/heart+disease).

In [26]:
!pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


This ``ucimlrepo`` package provides a nice interface for accessing their datasets.

In [27]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 
  
# variable information 
heart_disease.variables


,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,years,no
1,sex,Feature,Categorical,Sex,None,None,no
2,cp,Feature,Categorical,None,None,None,no
3,trestbps,Feature,Integer,None,resting blood pressure (on admission to the ho...,mm Hg,no
4,chol,Feature,Integer,None,serum cholestoral,mg/dl,no
5,fbs,Feature,Categorical,None,fasting blood sugar > 120 mg/dl,None,no
6,restecg,Feature,Categorical,None,None,None,no
7,thalach,Feature,Integer,None,maximum heart rate achieved,None,no
8,exang,Feature,Categorical,None,exercise induced angina,None,no
9,oldpeak,Feature,Integer,None,ST depression induced by exercise relative to ...,None,no


Here I remove the missing values from the features and labels.

In [28]:
bad = X.isna().any(axis=1)
X = X[~bad]
y = y[~bad]

Finally I convert the DataFrames to numpy arrays.

In [29]:
X = X.values
y = y.values.flatten()

The classification target is a number from 0-4 indicating the severity of heart disease.  Let's try fitting a linear model.

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
model = LogisticRegression().fit(X,y)

/Users/ishaansathaye/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
model.score(X,y)

0.6094276094276094

### Exercises

1. Compute the $\mathbf{z}$ values for the classifier manually, i.e. compute

$$\mathbf{W}\mathbf{X}+\mathbf{b}.$$

*Hints*: 
- Use `.shape` to get the shape of a Numpy matrix.
- ``@`` is the matrix multiplication operator in Numpy
- The actual computation will be a little different from what is written above.  You will need to use a matrix transpose which is `.T` in Numpy.

In [33]:
z = model.intercept_ + X @ model.coef_.T
z

array([[ 1.02965774,  0.445847  , -0.31956951, -0.33766716, -0.81826808],
       [-0.2035263 ,  0.06119195,  0.22316776,  0.39085182, -0.47168522],
       [-1.273311  ,  0.49377546,  0.50156916,  0.36806635, -0.09009997],
       ...,
       [-0.77804156,  0.37042732,  0.24356298,  0.43609437, -0.2720431 ],
       [-1.11470582,  0.53177271,  0.16409581,  0.64463538, -0.22579808],
       [ 3.65847204,  0.5625633 , -1.13349187, -1.59795482, -1.48958864]])

Print out the $\mathbf{z}$ values for the first example in the dataset and the first label.   Determine if the classifier is correctly classifying the first example in the dataset.

In [37]:
# z values
print(z[0])

# true label
print(y[0])

# predicted label using highest z value
print(z[0].argmax())

[ 1.02965774  0.445847   -0.31956951 -0.33766716 -0.81826808]
0
0


Classifer is classifying the first example correctly since the predicted class is 0 and the actual class is 0.

2. Use ``sklearn.model_selection.train_test_split`` to split ``X`` and ``y`` into 90% train and 10% test splits.  Note that this should be done in a single call to ``train_test_split``.

*Note*: Pass ``random_state=42`` to ``train_test_split`` to ensure you get the same result from random shuffling each time.


In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Fit the model to the training split and calculate accuracy on the test split.  How does it compare to the previous accuracy value (when the model was trained and evaluated on the same data)?

In [38]:
model = LogisticRegression().fit(X_train,y_train)

model.score(X_test,y_test)

/Users/ishaansathaye/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6666666666666666

The model does a little better on the test split than the training split.  This is a good sign that the model is generalizing somewhat well.

3. Run $k$-fold cross validation with $k=5$ and interpret the results (see `sklearn.model_selection.cross_val_score`).

In [39]:
from sklearn.model_selection import cross_val_score

cross_val_score(LogisticRegression(), X, y, cv=5)

/Users/ishaansathaye/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ishaansathaye/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

array([0.6       , 0.6       , 0.52542373, 0.55932203, 0.59322034])

The results of the cross_val_score show the accuracies for each fold. The average of these accuracies was around 57% which is not that great.